In [ ]:
# Install necessary modules
pip install pandas, pandas_gbq

In [ ]:
# Step 1: Import necessary modules
import pandas as pd
from pandas_gbq import read_gbq
from datetime import datetime, timedelta

In [ ]:
# Step 2: Extract data from BigQuery and store in respective pandas dataframes
bq_project_id = 'BIGQUERY PROJECT ID'
bq_dataset_id = 'BIGQUERY DATASET ID'
transactions_bq_table_id = 'TRANSACTIONS' # table id for transactions table
services_transactions_bq_table_id = 'SERVICES TRANSACTIONS' # table id for service_transactions table
users_bq_table_id = 'USERS' # table id for users table
services_bq_table_id = 'SERVICES' # table id for services table

# transactions
transactions_bq_query = f"SELECT user_email, amount, id, date(date_field) as date_field, transaction_type, transaction_status, country FROM `{bq_project_id}.{bq_dataset_id}.{transactions_bq_table_id}`"
transactions = read_gbq(transactions_bq_query, project_id=bq_project_id, dialect='standard')

# services_transactions
services_transactions_bq_query = f"SELECT id, amount, service_id, transaction_id FROM `{bq_project_id}.{bq_dataset_id}.{services_transactions_bq_table_id}`"
services_transactions = read_gbq(services_transactions_bq_query, project_id=bq_project_id, dialect='standard')

# users
users_bq_query = f"SELECT email, phone_number, name FROM `{bq_project_id}.{bq_dataset_id}.{users_bq_table_id}`"
users = read_gbq(users_bq_query, project_id=bq_project_id, dialect='standard')

# services
services_bq_query = f"SELECT name, id FROM `{bq_project_id}.{bq_dataset_id}.{services_bq_table_id}`"
services = read_gbq(services_bq_query, project_id=bq_project_id, dialect='standard')


In [ ]:
# Step 3: Load the dataframes into memory
transactions = pd.DataFrame(transactions)  
services_transactions = pd.DataFrame(services_transactions)
users = pd.DataFrame(users)
services = pd.DataFrame(services)

In [ ]:

def filter_service_data(
    transactions, services_transactions, users, services,
    service_name="cleaning", specific_date=None, num_days=None
):
    
    # Convert email to lowercase for joins
    transactions["user_email"] = transactions["user_email"].str.lower()
    users["email"] = users["email"].str.lower()

    # Merge dataframes
    merged_df = (
        transactions
        .merge(services_transactions, left_on="id", right_on="transaction_id", how="left")
        .merge(users, left_on="user_email", right_on="email", how="left")
        .merge(services, left_on="service_id", right_on="id", how="left")
    )

    # Filter for Nigerian records, successful transactions, and specific payment types
    filtered_df = merged_df[
        (merged_df["country"] == "NG") &
        (merged_df["transaction_status"] == "successful") &
        (merged_df["transaction_type"].str.lower().isin([
            "one-off", "subscription", "subscription_renewal", "reconfiguration"
        ])) &
        (merged_df["name_y"].isin(["cleaning", "laundry", "meal"]))
    ].copy()  # Create an explicit copy of the dataframe here

    # Compute order_count and customer_ltv (Using .loc now)
    filtered_df.loc[:, "order_count"] = filtered_df.groupby(["user_email", "name_y"])["id"].transform("count")
    filtered_df.loc[:, "customer_ltv"] = filtered_df.groupby(["user_email", "name_y"])["amount_y"].transform("sum")

    # Deduplicate to retain only the most recent record per user and service
    filtered_df["created_at"] = pd.to_datetime(filtered_df["created_at"])
    filtered_df = (
        filtered_df
        .sort_values(by=["user_email", "name_y", "created_at"], ascending=[True, True, False])
        .drop_duplicates(subset=["user_email", "name_y"], keep="first")
    )

    # Rename columns for clarity
    filtered_df.rename(columns={
        "name_x": "name",
        "name_y": "service_name",
        "amount_x": "last_transaction_amount",
        "created_at": "last_transaction_date"
    }, inplace=True)

    # Apply service name filter
    filtered_df = filtered_df[filtered_df["service_name"] == service_name]

    # Apply date filters
    if specific_date:
        specific_date = pd.to_datetime(specific_date)
        filtered_df = filtered_df[filtered_df["last_transaction_date"] <= specific_date]
    elif num_days:
        cutoff_date = datetime.now() - timedelta(days=num_days)
        filtered_df = filtered_df[filtered_df["last_transaction_date"] <= cutoff_date]

    # Select final columns for output
    result_df = filtered_df[[
        "name", "email", "phone_number", "last_transaction_date", "last_transaction_amount",
        "customer_ltv", "order_count", "service_name"
    ]]
    result_df = result_df.sort_values(by= ["last_transaction_date"], ascending=[False])
    # Format last_transaction_date to just date
    result_df['last_transaction_date'] = result_df['last_transaction_date'].dt.strftime('%Y-%m-%d')

    result_df = result_df.reset_index(drop=True)


    # Dynamically generate filename
    if specific_date:
        filename = f"churned_{service_name}_customers_{specific_date}.csv"
    elif num_days:
        filename = f"churned_{service_name}_customers_last_{num_days}_days.csv"
    else:
        filename = f"churned_{service_name}_customers.csv"  # Default filename

    result_df.to_csv(filename, index=False)  # Export to CSV

    return result_df

In [ ]:

# Sample code
result = filter_service_data(
     transactions, services_transactions, users, services,
     service_name="cleaning", specific_date= '2024-10-31', num_days= None
 )

result

In [ ]:
result.to_csv("result.csv", index = False)